In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input # A pretrained Model is used - MobileNet  
# from tensorflow.keras.models import Model # Functional API 

import tensorflow as tf

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm.notebook import tqdm


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers.legacy import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#path for training and validation dataset
train_path = "./data/train"
test_path = "./data/validation"

## Data Preprocessing


In [ ]:
def imageDataGenerator(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
#training dataframe
train_data = pd.DataFrame()
train_data['image'], train_data['label'] = imageDataGenerator(train_path)
print(train_data)


#testing dataframe
test_data = pd.DataFrame()
test_data['image'], test_data['label'] = imageDataGenerator(test_path)
print(test_data)

In [ ]:
def featureExtraction(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale" )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
#generating features for training set
train_features = featureExtraction(train_data['image']) 
test_features = featureExtraction(test_data['image'])

x_train = train_features/255.0
x_test = test_features/255.0


In [ ]:
#preprocessing the prediction label - string to numeric value
encoder  = LabelEncoder()
encoder.fit(train_data['label'])

y_train = encoder.transform(train_data['label'])
y_test = encoder.transform(test_data['label'])

y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

## CNN Model

In [ ]:
N_EPOCH = 40 # 40 bigger network will benefit from extra training epochs

def complexCNN(img_rows, img_cols, img_ch):
    # Complex DNN model definition
    model = Sequential()

    # #convolutional layers
    # model.add(Conv2D(128, kernel_size=3, padding='same', input_shape=(img_rows, img_cols, img_ch)))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.4))

    
    # model.add(Conv2D(256, kernel_size=3, padding='same'))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.4))

    # model.add(Conv2D(512, kernel_size=3, padding='same'))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.4))
    
    # model.add(Conv2D(512, kernel_size=3, padding='same'))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.4))


    # model.add(Flatten())

    # #full connected layer with no hidden layer
    # model.add(Dense(512))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.4))
    # model.add(Dense(256))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.3))

    # #output layer
    # model.add(Dense(7))
    # model.add(Activation('softmax'))

    #convolutional layers
    model.add(Conv2D(32, kernel_size=3, padding='same', input_shape=(img_rows, img_cols, img_ch)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))


    model.add(Flatten())

    #full connected layer with no hidden layer
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    #output layer
    model.add(Dense(7))
    model.add(Activation('softmax'))

    return model

complex_model = complexCNN(48,48,1)

    
#optim = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
complex_model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])
complex_model.summary()

In [ ]:
# Applying Early stopping and Model Check points
earlyStop = EarlyStopping(monitor="val_accuracy", min_delta = 0.01, patience=5, verbose=1, mode="auto")

modelCheckpoint = ModelCheckpoint(filepath="./saved_models/best_model.keras", monitor="val_accuracy", verbose=1, save_best_only = True, mode="auto")

call_backs = [earlyStop, modelCheckpoint]

history = complex_model.fit(x_train, y_train, epochs= 30, batch_size=128, validation_data = (x_test, y_test), callbacks=call_backs, verbose=1)

In [ ]:
# Plotting the training and validation accuracy
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()